In [41]:
#trying out medical embeddings
from gensim.models import KeyedVectors
filename = 'wikipedia-pubmed-and-PMC-w2v.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [14]:
import os
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

osPath = os.path.abspath("Tokenizer.ipynb")
path = 'concept_assertion_relation_training_data/beth/txt/'
files = os.listdir(path)

In [17]:
stop_words = set(stopwords.words('english'))

goods = []
bads = []
for file in files:
    #filter
    if not 'record-' in file:
        continue
    fpath = path+file
    with open(fpath) as iFile:
        for line in iFile:
            words = line.split()
            for w in words: 
                if not w in stop_words: 
                    if w in model:
                        goods.append(w)
                    else:
                        bads.append(w)

In [22]:
len(goods),len(bads),len(goods)/len(bads)

(67593, 2392, 28.25794314381271)

In [42]:
import os
tpath='concept_assertion_relation_training_data/beth/txt/'
cpath='concept_assertion_relation_training_data/beth/concept/'

In [43]:
tlist = [ f.split('.')[0] for f in os.listdir(tpath) if len(f.split('.')[0])>0 ]

In [44]:
#init
allrec = {}
for fname in tlist:
    f=open(tpath+fname+'.txt')
    rec = []
    for l in f:
        line = []
        for w in l.split():
            line.append([w.lower(),'O'])
        rec.append(line)
    allrec[fname] = rec

In [45]:
import re
for fname in tlist:
    f = open(cpath+fname+'.con')
    for i,l in enumerate(f):
        p = l.split('||')[0].split('"')[-1].strip()
        no = int(p.split()[0].split(':')[0])-1
        s = int(p.split()[0].split(':')[1])
        e = int(p.split()[1].split(':')[1])
        try:    
            for j in range(s,e+1):
                if s==j:
                    allrec[fname][no][j][1] = 'B'
                else:
                    allrec[fname][no][j][1] = 'I'
        except IndexError:
            print(fname,no,allrec[fname][no],s,e)
            break

In [47]:
for i,line in enumerate(allrec['record-176']):
    print(i+1,line,'\n')

1 [['admission', 'O'], ['date', 'O'], [':', 'O']] 

2 [['2016-08-08', 'O']] 

3 [['discharge', 'O'], ['date', 'O'], [':', 'O']] 

4 [['2016-08-15', 'O']] 

5 [['date', 'O'], ['of', 'O'], ['birth', 'O'], [':', 'O']] 

6 [['1978-08-02', 'O']] 

7 [['sex', 'O'], [':', 'O']] 

8 [['f', 'O']] 

9 [['service:', 'O']] 

10 [['procedure', 'O'], [':', 'O']] 

11 [['cadaveric', 'B'], ['pancreas', 'I'], ['transplantation', 'I'], ['.', 'O']] 

12 [['history', 'O'], ['of', 'O'], ['present', 'O'], ['illness', 'O'], [':', 'O']] 

13 [['the', 'O'], ['patient', 'O'], ['is', 'O'], ['a', 'O'], ['37', 'O'], ['year', 'O'], ['old', 'O'], ['lady', 'O'], ['with', 'O'], ['type', 'B'], ['1', 'I'], ['diabetes', 'I'], ['mellitus', 'I'], [',', 'O'], ['who', 'O'], ['is', 'O'], ['four', 'O'], ['months', 'O'], ['postcadaveric', 'B'], ['kidney', 'I'], ['transplantation', 'I'], ['and', 'O'], ['now', 'O'], ['has', 'O'], ['good', 'O'], ['graft', 'O'], ['function', 'O'], ['.', 'O']] 

14 [['she', 'O'], ['presents', 'O'], 

In [54]:
allre = allrec.copy()
for rk in allrec.keys():
    for i,line in enumerate(allrec[rk]):
        for j,word in enumerate(line):
            if word[0] not in model:
                del allre[rk][i][j]

In [55]:
allrec = allre
for rk in allrec.keys():
    for i,line in enumerate(allrec[rk]):
        for j,word in enumerate(line):
            if word[0] not in model:
                print(word)

In [63]:
recs = []
for rk in allrec.keys():
    for i,line in enumerate(allrec[rk]):
        li = []
        tag = []
        for j,word in enumerate(line):
            li.append(word[0])
            tag.append(word[1])
        recs.append((li,tag))
allrec = recs

In [64]:
import pickle
file = open('dat.pkl', 'wb')
pickle.dump(allrec, file)
file.close()